# SENTIMENT ANALYSIS USING RECURRENT NEURAL NETWORKS & EMBEDDINGS

_**Building recurrent neural networks and a model with pretrained embeddings for sentiment analysis on IMDb dataset and comparing their performance.**_

In [ ]:
# Imports required packages

import os
import tensorflow as tf

# import tensorflow_hub as tfhub

## Data Acquisition & Preprocessing

This experiment uses a large movie review dataset called IMDb dataset (https://ai.stanford.edu/~amaas/data/sentiment/) that contains English reviews for 50,000 movies - 25,000 for training and 25,000 for testing along with single binary target for each review indicating whether it is positive (1) or negative (0). Approximate download size is 80 megabytes (MB). The details of the dataset is available at https://www.tensorflow.org/datasets/catalog/imdb_reviews.

In [2]:
dataset_dir = "./../data/imdb_reviews_v1"

In [ ]:
train_set, val_set = tf.keras.utils.text_dataset_from_directory(
    os.path.join(dataset_dir, "train"),     # Path to train directory to load positive and negative samples
    labels= ...,            # Set it to "inferred" to infer labels from the name of the directories
    label_mode= ...,        # Set to "binary" to indicate it a binary classfication task
    class_names= ...,       # Set to a list with values "neg" and "pos" to read these subdirectories from and not from non-labeled "unsup"
    batch_size= ...,        # Set batch size to 32 - the default size
    shuffle= ...,           # Set it to `True` to shuffle the samples as part of best practice for model training
    seed= ...,              # Set to any integer such as 42 to enable reproducibility for the sequences of samples across classes and splits
    validation_split= ...,  # Set to to 0.2 as an indicator to seperate 20% of the training samples to be used as validation set
    subset= ...             # Set it to "both" for both train and validation dataset to be return to work with in subsequent steps
    )

# Pass the same arguments to the respective parameters for test set, too
test_set = tf.keras.utils.text_dataset_from_directory(
    os.path.join(dataset_dir, "test"), 
    labels= ...,
    label_mode= ...,
    class_names= ...,
    batch_size= ...,
    seed= ...,
    )

In [ ]:
# Pass `tf.data.AUTOTUNE` as argument to `prefetch` methods below to parallelize data transformation. This will help 
# overlapping the data preprocessing for step s+1 and while the model performs training at step s to save time.

train_set = train_set.prefetch( ... )
val_set = val_set.prefetch( ... )
test_set = test_set.prefetch( ... )

In [ ]:
# Reads few of samples by taking them randomly from the train set

for reviews, labels in train_set.take(1):
    for i in range(0, 5):
        print(f"Review: {reviews[i].numpy().decode("utf-8")}\nLabel: {labels[i]}\n")

## Modeling
_First models with training embedding - both with and without masking and then models with pretrained embeddings._

**Creating a Tokenizer**

 It prepares a tokenizer to tokenize text at the word level. It consideres English as text language.

In [ ]:
# Limits vocabulary to 1000 words: 998 tokens for frequent words plus
# one token for padding and one for unknown words
vocabulary_size = 1000

# Initialize TextVectorization layer to tokenize the input text by passing the following arguments
text_vectorizer_layer = tf.keras.layers.TextVectorization(
    max_tokens= ...,    # Set to the size of vocabulary. Effective number of tokens is (max_tokens - 1 - (1 if output_mode == "int" else 0))
    standardize= ...,   # Set it to 'lower_and_strip_punctuation' to lower case input string and to remove all punctuations
    split= ...,         # Set it to "whitespace" to split input text on
    output_mode= ...,   # Set it to "int" to output integer indices - one integer index per split string token. (0 is reserved for masked locations)
    )

# The text vectorizer gets adapted by passing train set through it so that
# it can tokenize the input text during training and inferencing
text_vectorizer_layer.adapt(
    train_set.map(lambda reviews, labels: reviews))

In [ ]:
# Passes a sample text to check if text vectorizer works by observing 
# the output integer indices corresponding to the words in the input string
text_vectorizer_layer("This is a sample positive movie review.")

### Modeling with Trainable Word Embeddings

#### Modeling without Masking
`TextVectorization` layer pads shorter sequences with padding token (with ID 0) to make them as long as the longest sequence in the batch.

In [ ]:
# Defines the size of the embedding 
embedding_size = 128

tf.random.set_seed(42)        # Sets the global random seed for operations that rely on a random seed

# Create a sequential model with the specific layers as mentioned
model = tf.keras.Sequential([
    ...,    # Set to `text_vectorizer_layer` as a Vectorizer layer (already adapted) to first tokenize input string

    ...,    # An `tf.keras.layers.Embedding` layer (with parameter `input_dim` set to vocabulary size
            # and `output_dim` set to embedding size) as trainable embedding that learns to represent
            # each token into a dense vector of fixed size

    ...,    # A `tf.keras.layers.GRU` layer (with parameter `unit` set to 128 as output units and 
            # `return_sequences` set to `False` to return last output in output sequence (instead of full sequence)

    ...     # Regular dense layer `tf.keras.layers.Dense` with parameter 1 as output unit (for binary classification)
            # and `activation` set to "sigmoid"
])

In [ ]:
# Compile the model with "nadam" as optimizer, "binary_crossentropy" as loss function and 
# ["accuracy"] as list of metrics
model.compile(optimizer= ..., loss= ..., metrics= ...)

# Fit the model by passing train set in the first parameter, validation set and 5 as number of epochs in
# the remaining two named parameters
history = model.fit( ..., validation_data= ..., epochs= ...)

Observe the model's prediction performance on the validation set and analyze it.

#### Modeling with Masking
_Masking is enabled at the embedding layer for it to propagate this information to all downstream layers to skip the padding tokens so that prediction performance of the model improves._

In [ ]:
# Create another sequential model as instructed above, but this time pass argument `True` to parameter
# `mask_zero` to function `tf.keras.layers.Embedding` to masks padding tokens (whose ID is 0)
model = tf.keras.Sequential([
    ...,
    ...,
    ...,
    ...])

In [ ]:
# Use the same settings to compile and fit the model once again

model.compile(optimizer= ..., loss= ..., metrics= ...)

history = model.fit( ..., validation_data= ..., epochs=)

Observe the model's prediction performance on the validation set post masking and analyze it..

In [ ]:
# Evaluate model over test set
model_test_performance = model.evaluate(test_set)

print(f"Test Performance (accuracy): {model_test_performance[1] * 100:.1f}%")

### [OPTIONAL] Modeling with Pretrained Embeddings
_Experimenting with pretrained sentence-level embeddings for classification task._

**Known Issues**

1. One of the concerns of this experiment is that the pretrained embedding is non-tunable [refer model summary to find that the trainable parameters is shown as zero] even though the layer that wraps the embedding was set as trainable [refer wrapper class `USE_Embedding`]. Fixing that problem could improve the prediction performance (accuracy) of the model.

2. While training the USE pretrained embeddings based model in Google Colab on T4 GPU with TensorFlow version 2.18.0, XLA compiler that accelerates computations on GPU raised TensorFlow graph execution error. Since the error message highlighted "T=DT_STRING" it is highly likely that the string input to the USE embedding layer is causing the problem. The XLA compiler might not have the necessary kernels to handle string data on the GPU. Fixing this problem could further improve model training time performance.

One of the pretrained sentense encoders called _Universal Sentence Encoder_ (USE) from TensorFlow Hub was used. It is already trained over a large corpus and helps in finding sentence-level meaning similarity enenbling better performance for downstream classification tasks. The model could also be fine-tuned over the task in hand to improve prediction performance.

In [ ]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"    # Location of USE to download from

In [ ]:
# class USE_Embedding(tf.keras.layers.Layer):
#     """
#     Defines a custom layer to wrap USE. This is primarily due to the fact that newer TensorFlow 
#     frameworks [version 2.15+] do not anymore support saved model to be used as layer in Keras sequential model.
#     """
#     def __init__(self, module_url, **kwargs):           # Used as constructor
#         super(USE_Embedding, self).__init__(**kwargs)   # Calls initialization method of the parent class

#         # Class variable 'embed' is set to embedding layer
#         # NOTE: Downloading the below module which is around 1 GB in size may take few minutes to complete
#         self.embed = tfhub.KerasLayer(module_url, trainable=True, input_shape=[], dtype=tf.string)

#     def call(self, inputs):                             # Gets called when an instance behaves like a function
#         return self.embed(inputs)                       # Returns the embeddings against the inputs

In [ ]:
# # Creates a functional model containing USE as a layer

# input = tf.keras.Input(shape=(), dtype=tf.string)               # shape=() indicates input sequence length is unknown or may vary

# embedding = USE_Embedding(module_url)(input)                    # Pretrained embedding layer to receive input

# dense = tf.keras.layers.Dense(64, activation="relu")(embedding) # Regular dense layer

# output = tf.keras.layers.Dense(1, activation="sigmoid")(dense)  # Regular one-output dense layer for binary classification

# model = tf.keras.Model(inputs=input, outputs=output)            # Groups layers as a model for training and/or inference


In [ ]:
# Shows the model summary
# model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ use__embedding_2                │ (None, 512)            │             0 │
│ (USE_Embedding)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,897 (128.50 KB)

 Trainable params: 32,897 (128.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# # Compiles the mode and trains it

# model.compile(optimizer="nadam", loss="binary_crossentropy", metrics=["accuracy"])

# history = model.fit(train_set, 
#                     validation_data=val_set, 
#                     epochs=10, 
#                     callbacks=[
#                         tf.keras.callbacks.ModelCheckpoint(
#                             "./model_weights/my_universal_sentence_encoder_model.weights.h5", 
#                             monitor='val_accuracy', 
#                             save_best_only=True, 
#                             save_weights_only=True)
#                         # tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', restore_best_weights=True)]
#                     ])

# # Loads back the weights of the best model
# model.load_weights("./model_weights/my_universal_sentence_encoder_model.weights.h5")

Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 25s 31ms/step - accuracy: 0.7967 - loss: 0.4675 - val_accuracy: 0.8504 - val_loss: 0.3283
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.8605 - loss: 0.3290 - val_accuracy: 0.8548 - val_loss: 0.3232
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.8628 - loss: 0.3197 - val_accuracy: 0.8496 - val_loss: 0.3250
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.8683 - loss: 0.3183 - val_accuracy: 0.8520 - val_loss: 0.3222
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.8680 - loss: 0.3125 - val_accuracy: 0.8564 - val_loss: 0.3200
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.8674 - loss: 0.3120 - val_accuracy: 0.8520 - val_loss: 0.3168
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.8727 - loss: 0.3075 - val_accuracy: 0.8580 - val_loss: 0.3164
Epoch 8/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.8734 - loss: 0.3010 - 

The validation accuracy of the above model has reached to 86%.

In [ ]:
# # Evaluates model over test set
# model_test_performance = model.evaluate(test_set)

# print("Test Performance [Accuracy]: {:.1f}%".format(model_test_performance[1] * 100))

## Observations

- How was the labels inferred subdirectories without reading from other directories of non-interest?

- What were the major considerations to load data using TensorFlow dataset?

- Why was a tokenizer used? How did it preprocess the input string? How length of all input string were made same?

- Why did the model performed poorly when masking was not enabled? Change change was made into model to improve prediction performance?